# News Modeling

Topic modeling involves **extracting features from document terms** and using
mathematical structures and frameworks like matrix factorization and SVD to generate **clusters or groups of terms** that are distinguishable from each other and these clusters of words form topics or concepts

Topic modeling is a method for **unsupervised classification** of documents, similar to clustering on numeric data

These concepts can be used to interpret the main **themes** of a corpus and also make **semantic connections among words that co-occur together** frequently in various documents

Topic modeling can help in the following areas:
- discovering the **hidden themes** in the collection
- **classifying** the documents into the discovered themes
- using the classification to **organize/summarize/search** the documents

Frameworks and algorithms to build topic models:
- Latent semantic indexing
- Latent Dirichlet allocation
- Non-negative matrix factorization

## Latent Dirichlet Allocation (LDA)
The latent Dirichlet allocation (LDA) technique is a **generative probabilistic model** where each **document is assumed to have a combination of topics** similar to a probabilistic latent semantic indexing model

In simple words, the idea behind LDA is that of two folds:
- each **document** can be described by a **distribution of topics**
- each **topic** can be described by a **distribution of words**

### LDA Algorithm

- 1. For each document, **randomly initialize each word to one of the K topics** (k is chosen beforehand)
- 2. For each document D, go through each word w and compute:
    - **P(T |D)** , which is a proportion of words in D assigned to topic T
    - **P(W |T )** , which is a proportion of assignments to topic T over all documents having the word W
- **Reassign word W with topic T** with probability P(T |D)´ P(W |T ) considering all other words and their topic assignments

![LDA](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/LDA.png)

### Steps
- Install the necessary library
- Import the necessary libraries
- Download the dataset
- Load the dataset
- Pre-process the dataset
    - Stop words removal
    - Email removal
    - Non-alphabetic words removal
    - Tokenize
    - Lowercase
    - BiGrams & TriGrams
    - Lemmatization
- Create a dictionary for the document
- Filter low frequency words
- Create an Index to word dictionary
- Train the Topic Model
- Predict on the dataset
- Evaluate the Topic Model
    - Model Perplexity
    - Topic Coherence
- Visualize the topics

### Install the necessary library

In [1]:
! pip install pyLDAvis gensim spacy

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyLDAvis-3.4.0-py3-none-any.whl (2.6 MB)
     --------------------------------------- 24.0/24.0 MB 11.5 MB/s eta 0:00:00
     --------------------------------------- 12.2/12.2 MB 15.9 MB/s eta 0:00:00
     -------------------------------------- 133.1/133.1 kB 8.2 MB/s eta 0:00:00
     ---------------------------------------- 92.7/92.7 kB ? eta 0:00:00
  Using cached funcy-1.18-py2.py3-none-any.whl (33 kB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
     ---------------------------------------- 94.7/94.7 kB ? eta 0:00:00
  Using cached wasabi-1.1.1-py3-none-any.whl (27 kB)
     ---------------------------------------- 1.5/1.5 MB 15.5 MB/s eta 0:00:00
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
     ---------------------------------------- 2.1/2.1 MB 19.3 M

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import the libraries

In [2]:
import nltk
! nltk.download('stopwords')

'nltk.download' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
! pip install gensim
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import gensim


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


### Download the dataset
Dataset: https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

#### 20-Newsgroups dataset
- 11K newsgroups posts
- 20 news topics

In [4]:
! wget https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

--2023-03-20 11:45:15--  https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23237087 (22M) [text/plain]
Saving to: 'newsgroups.json.5'

     0K .......... .......... .......... .......... ..........  0% 6.74M 3s
    50K .......... .......... .......... .......... ..........  0% 58.3M 2s
   100K .......... .......... .......... .......... ..........  0% 12.4M 2s
   150K .......... .......... .......... .......... ..........  0% 27.6M 2s
   200K .......... .......... .......... .......... ..........  1% 74.9M 1s
   250K .......... .......... .......... .......... ..........  1% 6.70M 2s
   300K .......... .......... .......... .......... ..........  1%  123M 1s
   350K .......... ......

  9150K .......... .......... .......... .......... .......... 40% 29.5M 1s
  9200K .......... .......... .......... .......... .......... 40% 21.5M 1s
  9250K .......... .......... .......... .......... .......... 40% 18.3M 1s
  9300K .......... .......... .......... .......... .......... 41%  143M 1s
  9350K .......... .......... .......... .......... .......... 41% 24.7M 1s
  9400K .......... .......... .......... .......... .......... 41% 32.7M 1s
  9450K .......... .......... .......... .......... .......... 41% 16.5M 1s
  9500K .......... .......... .......... .......... .......... 42%  120M 1s
  9550K .......... .......... .......... .......... .......... 42% 23.2M 1s
  9600K .......... .......... .......... .......... .......... 42% 21.8M 1s
  9650K .......... .......... .......... .......... .......... 42%  236M 1s
  9700K .......... .......... .......... .......... .......... 42% 42.1M 1s
  9750K .......... .......... .......... .......... .......... 43% 10.6M 1s
  9800K ....

### Load the dataset

In [5]:
import pandas as pd

df = pd.read_json('newsgroups.json')
print(df.head())

                                             content  target  \
0  From: lerxst@wam.umd.edu (where's my thing)\nS...       7   
1  From: guykuo@carson.u.washington.edu (Guy Kuo)...       4   
2  From: twillis@ec.ecn.purdue.edu (Thomas E Will...       4   
3  From: jgreen@amber (Joe Green)\nSubject: Re: W...       1   
4  From: jcm@head-cfa.harvard.edu (Jonathan McDow...      14   

            target_names  
0              rec.autos  
1  comp.sys.mac.hardware  
2  comp.sys.mac.hardware  
3          comp.graphics  
4              sci.space  


### Preprocess the data

In [6]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chail\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chail\AppData\Roaming\nltk_data...


True

In [8]:
df = pd.read_json('newsgroups.json')

In [9]:
# Preprocess the text column
df['preprocessed_text'] = df['content'].apply(lambda x: x.lower())
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
stop_words = set(stopwords.words('english'))
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
lemmatizer = WordNetLemmatizer()
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [10]:
df.head()

,content,target,target_names,preprocessed_text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos,lerxstwamumdedu wheres thing subject car nntpp...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware,guykuocarsonuwashingtonedu guy kuo subject si ...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware,twillisececnpurdueedu thomas e willis subject ...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics,jgreenamber joe green subject weitek p organiz...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space,jcmheadcfaharvardedu jonathan mcdowell subject...


### Email Removal

In [11]:
import re

In [12]:
# Defininig a regular expression for matching email addresses
email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Removing email addresses from the preprocessed text
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: re.sub(email_regex, '', x))
df.head()

,content,target,target_names,preprocessed_text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos,lerxstwamumdedu wheres thing subject car nntpp...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware,guykuocarsonuwashingtonedu guy kuo subject si ...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware,twillisececnpurdueedu thomas e willis subject ...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics,jgreenamber joe green subject weitek p organiz...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space,jcmheadcfaharvardedu jonathan mcdowell subject...


### Newline Removal

In [13]:
# Removing newlines from the preprocessed text
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: x.replace('\n', ' '))
df.head()

,content,target,target_names,preprocessed_text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos,lerxstwamumdedu wheres thing subject car nntpp...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware,guykuocarsonuwashingtonedu guy kuo subject si ...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware,twillisececnpurdueedu thomas e willis subject ...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics,jgreenamber joe green subject weitek p organiz...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space,jcmheadcfaharvardedu jonathan mcdowell subject...


### Single Quotes Removal

In [14]:
# Removing single quotes from the preprocessed text
df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: x.replace("'", ""))
df.head()

,content,target,target_names,preprocessed_text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos,lerxstwamumdedu wheres thing subject car nntpp...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware,guykuocarsonuwashingtonedu guy kuo subject si ...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware,twillisececnpurdueedu thomas e willis subject ...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics,jgreenamber joe green subject weitek p organiz...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space,jcmheadcfaharvardedu jonathan mcdowell subject...


### Tokenize
- Create **sent_to_words()** 
    - Use **gensim.utils.simple_preprocess**
    - Use **generator** instead of an usual function

In [15]:
from gensim.utils import simple_preprocess

In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True)) #True means removes punctuation
#usage example
data = df['preprocessed_text'].values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:5])

[['lerxstwamumdedu', 'wheres', 'thing', 'subject', 'car', 'nntppostinghost', 'racwamumdedu', 'organization', 'university', 'maryland', 'college', 'park', 'line', 'wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'door', 'sport', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'door', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please', 'email', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst'], ['guy', 'kuo', 'subject', 'si', 'clock', 'poll', 'final', 'call', 'summary', 'final', 'call', 'si', 'clock', 'report', 'keywords', 'articleid', 'organization', 'university', 'washington', 'line', 'nntppostinghost', 'fair', 'number', 'brave', 'soul', 'upgraded', 'si', 'clock', 'oscillator', 'shared', 'experience', 'poll', 'please', 'send', 'brief', 'message', 'detailing', 'experie

### Stop words Removal
- Extend the stop words corpus with the following words
    - from
    - subject
    - re
    - edu
    - use

In [17]:
from gensim.parsing.preprocessing import STOPWORDS

In [18]:
# Defining additional stop words
CUSTOM_STOPWORDS = STOPWORDS.union(set(['from', 'subject', 're', 'edu', 'use']))

def remove_stopwords(texts):
    return [[word for word in doc if word not in CUSTOM_STOPWORDS] for doc in texts]

# Usage example:
data_words_nostops = remove_stopwords(data_words)
print(data_words_nostops[:5])

[['lerxstwamumdedu', 'wheres', 'thing', 'car', 'nntppostinghost', 'racwamumdedu', 'organization', 'university', 'maryland', 'college', 'park', 'line', 'wondering', 'enlighten', 'car', 'saw', 'day', 'door', 'sport', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'spec', 'year', 'production', 'car', 'history', 'info', 'funky', 'looking', 'car', 'email', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst'], ['guy', 'kuo', 'si', 'clock', 'poll', 'final', 'summary', 'final', 'si', 'clock', 'report', 'keywords', 'articleid', 'organization', 'university', 'washington', 'line', 'nntppostinghost', 'fair', 'number', 'brave', 'soul', 'upgraded', 'si', 'clock', 'oscillator', 'shared', 'experience', 'poll', 'send', 'brief', 'message', 'detailing', 'experience', 'procedure', 'speed', 'attained', 'cpu', 'rated', 'speed', 'add', 'card', 'adapter', 'heat', 'sink', 'hour', 'usage', 'day', 'fl

#### remove_stopwords( )

In [19]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [20]:
def remove_stopwords(texts):

    return [[word for word in doc if word not in stop_words] for doc in texts if doc]

In [21]:
data_words_nostops = remove_stopwords(data_words)

### Bigrams
- Use **gensim.models.Phrases**
- 100 as threshold

In [22]:
from gensim.models.phrases import Phrases, Phraser

In [23]:
# Build the bigram models
bigram = Phrases(data_words_nostops, min_count=5, threshold=100)
bigram_mod = Phraser(bigram)

In [24]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

data_words_bigrams = make_bigrams(data_words_nostops)

In [25]:
print(data_words_nostops[:10])
print(data_words[:10])

[['lerxstwamumdedu', 'wheres', 'thing', 'car', 'nntppostinghost', 'racwamumdedu', 'organization', 'university', 'maryland', 'college', 'park', 'line', 'wondering', 'anyone', 'could', 'enlighten', 'car', 'saw', 'day', 'door', 'sport', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'door', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'car', 'please', 'email', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst'], ['guy', 'kuo', 'si', 'clock', 'poll', 'final', 'call', 'summary', 'final', 'call', 'si', 'clock', 'report', 'keywords', 'articleid', 'organization', 'university', 'washington', 'line', 'nntppostinghost', 'fair', 'number', 'brave', 'soul', 'upgraded', 'si', 'clock', 'oscillator', 'shared', 'experience', 'poll', 'please', 'send', 'brief', 'message', 'detailing', 'experience', 'procedure', 'to

#### make_bigrams( )

In [26]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

### Lemmatization
- Use spacy
    - Download spacy en model (if you have not done that before)
    - Load the spacy model

In [27]:
! pip install spacy
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 12.8/12.8 MB 18.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import spacy

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

#### lemmatizaton( )

In [32]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [33]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [34]:
print(data_lemmatized[:1])

[['s', 'thing', 'car', 'nntppostinghost', 'organization', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'email', 'thank', 'bring', 'neighborhood', 'lerxst']]


### Create a Dictionary

In [35]:
id2word = corpora.Dictionary(data_lemmatized)

### Create Corpus

In [36]:
texts = data_lemmatized

### Filter low-frequency words

In [37]:
id2word.filter_extremes(no_below=5, no_above=0.5)

### Create Index 2 word dictionary

In [38]:
id2word = corpora.Dictionary(data_lemmatized)

In [39]:
print(id2word)

Dictionary<55919 unique tokens: ['addition', 'body', 'bring', 'call', 'car']...>


### Build a News Topic Model

#### LdaModel
- **num_topics** : this is the number of topics you need to define beforehand
- **chunksize** : the number of documents to be used in each training chunk
- **alpha** : this is the hyperparameters that affect the sparsity of the topics
- **passess** : total number of training assess

In [40]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary

In [41]:
# Creating a dictionary
dictionary = Dictionary(data_lemmatized)

In [42]:
# Filtering out words that occur less than 10 documents
dictionary.filter_extremes(no_below=10, no_above=0.5)

In [43]:
# Creating a bag of words corpus
corpus = [dictionary.doc2bow(text) for text in data_lemmatized]

In [44]:
# Trainining the LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=20,
                     chunksize=100,
                     alpha='auto',
                     passes=10,
                     random_state=42)

### Print the Keyword in the 10 topics

In [45]:
for idx, topic in lda_model.show_topics(formatted=False, num_topics=10):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))
    print('\n')

Topic: 3 
Words: ['kill', 'soldier', 'village', 'armenian', 'war', 'attack', 'land', 'murder', 'people', 'turkish']


Topic: 14 
Words: ['space', 'center', 'launch', 'mission', 'agency', 'research', 'development', 'orbit', 'satellite', 'moon']


Topic: 15 
Words: ['image', 'earth', 'picture', 'object', 'search', 'scan', 'requirement', 'planet', 'surface', 'role']


Topic: 13 
Words: ['file', 'program', 'entry', 'format', 'nhl', 'session', 'decent', 'noise', 'correctly', 'att']


Topic: 10 
Words: ['window', 'driver', 'internet', 'application', 'screen', 'display', 'printer', 'manager', 'library', 'version']


Topic: 12 
Words: ['number', 'email', 'include', 'send', 'information', 'book', 'post', 'source', 'list', 'address']


Topic: 8 
Words: ['system', 'use', 'problem', 'run', 'set', 'card', 'bit', 'machine', 'help', 'work']


Topic: 7 
Words: ['evidence', 'reason', 'believe', 'people', 'claim', 'state', 'fact', 'case', 'exist', 'mean']


Topic: 6 
Words: ['get', 'nntppostinghost', 'n

## Evaluation of Topic Models
- Model Perplexity
- Topic Coherence

### Model Perplexity

Model perplexity is a measurement of **how well** a **probability distribution** or probability model **predicts a sample**

In [46]:
print('Perplexity:', lda_model.log_perplexity(corpus))

Perplexity: -11.923750230142153


### Topic Coherence
Topic Coherence measures score a single topic by measuring the **degree of semantic similarity** between **high scoring words** in the topic.

In [47]:
from gensim.models import CoherenceModel

In [48]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4931678054187249


### Visualize the Topic Model
- Use **pyLDAvis**
    - designed to help users **interpret the topics** in a topic model that has been fit to a corpus of text data
    - extracts information from a fitted LDA topic model to inform an interactive web-based visualization

In [49]:
!pip install pyLDAvis

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
import pyLDAvis.gensim_models
import gensim

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

C:\Users\chail\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.285719 -0.038661       1        1  33.459221
6     -0.247760  0.070893       2        1  14.519677
7     -0.214862 -0.041271       3        1  11.745034
8     -0.175826  0.175406       4        1   8.321233
12    -0.147177  0.123665       5        1   7.153751
0     -0.093588 -0.056818       6        1   2.835428
19    -0.007719 -0.301552       7        1   2.579385
16     0.055028 -0.064216       8        1   2.232365
4      0.084497  0.013853       9        1   2.186804
17    -0.014322  0.078585      10        1   1.899152
2      0.098383 -0.095446      11        1   1.841676
9      0.110554  0.112584      12        1   1.701173
3     -0.001243 -0.314738      13        1   1.644228
1      0.117398  0.036617      14        1   1.423215
18     0.096682  0.068377      15        1   1.421009
11     0.144573  0.007012      16        1   1.154954
13     0.129732  0.066212      17        1   1.071679
10     0.115482  0.075915      18        1   1.062075
14     0.090241  0.042683      19        1   1.027408
15     0.145647  0.040898      20        1   0.720533, topic_info=             Term         Freq        Total Category  logprob  loglift
298          file  2677.000000  2677.000000  Default  30.0000  30.0000
767       program  2744.000000  2744.000000  Default  29.0000  29.0000
123        people  7376.000000  7376.000000  Default  28.0000  28.0000
322         drive  3115.000000  3115.000000  Default  27.0000  27.0000
176        system  4935.000000  4935.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2545      preview    10.146728    11.435694  Topic20  -6.5659   4.8133
2620     approval    35.420680    46.231930  Topic20  -5.3157   4.6666
1875  observation    83.523460   228.762505  Topic20  -4.4579   3.9254
446        create   130.327958   961.547199  Topic20  -4.0130   2.9344
867      maintain    42.840070   390.516353  Topic20  -5.1256   2.7229

[891 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
429       9  0.993878    absolute
1119      3  0.998574      accept
75       10  0.294621      access
75       12  0.702558      access
1780     20  0.990388  accomplish
...     ...       ...         ...
34       11  0.008818        year
34       13  0.027149        year
213       1  0.998673         yet
524       6  0.088864       young
524       9  0.909033       young

[1203 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 8, 9, 13, 1, 20, 17, 5, 18, 3, 10, 4, 2, 19, 12, 14, 11, 15, 16])